In [ ]:
# Path: notebooks/SPVCNN_preprocessing/root_to_pickle.ipynb
# This is a script to convert root files to pickle files 
import uproot 
import numpy as np
import pandas as pd
from pathlib import Path
import awkward as ak
import os, sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.join("..", "src")))
from pathlib import Path

samples_folder = Path("../../samples")

In [ ]:
file_path = (
    samples_folder / "sample/n1000_npu50_IVF/vertex_n1000_npu50_IVF_04282023.root"
)

vertex_algo = "IVF"

In [ ]:
file_path.as_posix()

In [ ]:
file = uproot.open(file_path)

In [ ]:
file.keys()

In [ ]:
raw_data_dir = Path("./tmp")

In [ ]:
root_dir = uproot.open(file_path)

ML_tracks_tree = root_dir["ML_tracks"]

In [ ]:
ML_tracks_tree.keys()

In [ ]:
ML_tracks_tree['reco_trk_reco_vtx_idx'].array()

In [ ]:
.keys()

In [ ]:
reco_trk_reco_vtx_trackWeight = ML_tracks_tree['reco_trk_reco_vtx_trackWeight'].array()
reco_trk_reco_vtx_idx = ML_tracks_tree['reco_trk_reco_vtx_idx'].array()

In [ ]:
ak.count(ML_tracks_tree['reco_trk_reco_vtx_idx'].array(), axis = 2) - 1

In [ ]:
ML_tracks_tree['reco_trk_reco_vtx_idx']

In [ ]:
np.max(ML_tracks_tree['reco_trk_reco_vtx_idx'].array(), axis=1)

In [ ]:
np.argmax(ML_tracks_tree['reco_trk_reco_vtx_trackWeight'].array(), axis=2)

In [ ]:
reco_trk_prefix = "reco_trk_fit_"

ML_tracks_dict = {}
for k, v in ML_tracks_tree.items():
    if not k.startswith(reco_trk_prefix):
        continue
    ML_tracks_dict[k[len(reco_trk_prefix):]] = v.array(library="np")

ML_tracks_dict['truth_instance_label'] = ML_tracks_tree['reco_trk_truth_vtx_idx'].array()
reco_trk_reco_vtx_idx = ML_tracks_tree['reco_trk_reco_vtx_idx'].array()
reco_trk_reco_vtx_trackWeight = ML_tracks_tree['reco_trk_reco_vtx_trackWeight'].array()

ni = 0

# ML_tracks_dict['reco_instance_label'] = ML_tracks_tree['reco_trk_reco_vtx_idx'].array()
for n in tqdm(range(len(ML_tracks_tree['event_nr'].array()[0:1]))):
    ML_tracks_df = {k: ML_tracks_dict[k][n] for k in ML_tracks_dict.keys() if k != 'reco_instance_label'}
            
    truth_flat_event = pd.DataFrame(ML_tracks_df)

    truth_flat_event['truth_semantic_label'] = [1] * len(truth_flat_event)
    truth_flat_event.loc[(truth_flat_event['truth_instance_label'] == -999) | truth_flat_event['truth_instance_label'] == -1, 
                          'truth_semantic_label'] = 0
    
    # ML_tracks_dict['reco_instance_label']
    np.argmax(reco_trk_reco_vtx_trackWeight[n])
    truth_flat_event['reco_semantic_label'] = [1] * len(truth_flat_event)
    # truth_flat_event.loc[(truth_flat_event['truth_instance_label'] == -999) | truth_flat_event['truth_instance_label'] == -1, 
    #                       'truth_semantic_label'] = 0

    truth_flat_event['x0'] = truth_flat_event['d0'] * np.cos(truth_flat_event['phi'] )
    truth_flat_event['y0'] = truth_flat_event['d0'] * np.sin(truth_flat_event['phi'] )


    truth_flat_event.dropna(inplace=True) # For some cases the tracks has dummy values 

    # truth_flat_event.to_pickle(raw_data_dir / f'event_{n+ni:05}.pkl')
# ni += n + 1
    


In [ ]:
max_weight_idx = np.argmax(reco_trk_reco_vtx_trackWeight[n],axis=1)

In [ ]:
reco_trk_reco_vtx_idx[n][max_weight_idx]

In [ ]:
reco_trk_reco_vtx_idx[n]

In [ ]:
max_weight_idx

In [ ]:
ak.flatten(reco_trk_reco_vtx_idx[n][max_weight_idx], )

In [ ]:
truth_flat_event

In [ ]:
def root_to_pickle(root_data_path, raw_data_dir):
    if not raw_data_dir.exists():
        raw_data_dir.mkdir(parents=True, exist_ok=True)
    ni = 0
    for f in sorted(root_data_path.glob('*.root')):
        print(f)
        root_dir = uproot.open(f)
        truth_tree = root_dir['Truth_Vertex_PV_Selected']
        reco_tree = root_dir['Reco_Vertex']
        truth_jagged_dict = {}
        reco_jagged_dict = {}
        truth_prefix = 'reco_trk_fit_'
        reco_prefix = ''

        for k, v in truth_tree.items():
            if not k.startswith(truth_prefix):
                continue
            truth_jagged_dict[k[len(truth_prefix):]] = v.array()
        
        for k, v in reco_tree.items():
            if not k.startswith(reco_prefix):
                continue
            reco_jagged_dict[k[len(reco_prefix):]] = v.array()
        
        truth_jagged_dict['truth_vtxID'] = truth_jagged_dict.pop('vtxID')
        reco_jagged_dict['amvf_vtxID'] = reco_jagged_dict.pop('vtxID')

        for n in tqdm(range(len(truth_tree[0].array()))):
            truth_df_dict = {k: truth_jagged_dict[k][n] for k in truth_jagged_dict.keys()}
            reco_df_dict = {l: reco_jagged_dict[l][n] for l in reco_jagged_dict.keys()}
            
            truth_flat_event = pd.DataFrame(truth_df_dict)
            reco_flat_event = pd.DataFrame(reco_df_dict)
            truth_flat_event['truth_semantic_label'] = [1] * len(truth_flat_event)
            truth_flat_event['x0'] = truth_flat_event['d0'] * np.cos(truth_flat_event['phi'] )
            truth_flat_event['y0'] = truth_flat_event['d0'] * np.sin(truth_flat_event['phi'] )

            # Add truth vtx weights 
            # truth_vtx_sumpt2 = np.bincount(truth_flat_event['truth_vtxID'], weights=((1.0/truth_flat_event['qp'])**2))
            # truth_vtx_sumpt2 = truth_vtx_sumpt2 / truth_vtx_sumpt2.sum() # normalization 
            # truth_flat_event['truth_vtx_weight'] = truth_vtx_sumpt2[truth_flat_event['truth_vtxID']] # attach to the DataFrame
            
            # Attach amvf reco vtx labels to the flat event  
            # conterpart is pf_sem_labels and pf_ins_labels
            match_idx = get_match_idx(truth_flat_event, reco_flat_event)
            truth_flat_event['reco_AMVF_vtxID'] = match_idx
            truth_flat_event['reco_semantic_label'] = [1] * len(truth_flat_event)
            
            # For those vtx considered as outliers by amvf, set the reco_semantic_label to 0. 
            idx_not_found = truth_flat_event['reco_AMVF_vtxID'] == -1
            truth_flat_event.loc[idx_not_found,'reco_semantic_label'] = [0]*len(truth_flat_event.loc[idx_not_found,'reco_semantic_label'])
            truth_flat_event.dropna(inplace=True) # For some cases the 

            truth_flat_event.to_pickle(raw_data_dir / f'event_{n+ni:05}.pkl')
        ni += n + 1